<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](https://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# OpenEnv GRPO with trl

In this tutorial notebook, we're going to use Oumi to train an agentic model on an [OpenEnv](https://github.com/meta-pytorch/OpenEnv) Echo reinforcement learning (RL) environment with the GRPO algorithm. To achieve this, we use the trl library by Hugging Face with a custom rollout function to interact with the vLLM server and OpenEnv environment.

# 📋 Prerequisites

❗**NOTICE:** This notebook needs to be running on a machine with at least two GPUs.

## Oumi Installation

First, let's install the latest versions of Oumi, trl, and OpenEnv. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html).

In [ ]:
# !pip install uv && uv pip install "oumi[gpu] @ git+https://github.com/oumi-ai/oumi.git"
!pip install uv && uv pip install -e "..[gpu]"
!uv pip install git+https://github.com/meta-pytorch/OpenEnv.git
!uv pip install git+https://github.com/huggingface/trl.git

In [1]:
import os
from pathlib import Path

tutorial_dir = "openenv_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Disable warnings from HF.

# Start OpenEnv and vLLM servers

We need to run 2 servers in addition to the trl trainer. The OpenEnv server receives actions from the LLM and returns the updated state and reward. The vLLM server is used for inference, and updates it weights over training with the updated model weights from the trainer. We start these with separate subprocesses.

In [2]:
%%writefile $tutorial_dir/start_openenv_server.py

import os
import subprocess
import sys
import threading
import time
from pathlib import Path

import requests


def stream_output(pipe, prefix=""):
    """Stream output lines from subprocess pipe to stdout."""
    for line in iter(pipe.readline, ""):
        print(f"{prefix}{line}", end="")
    pipe.close()


print("⚡ Starting FastAPI server for Echo Environment...")

work_dir = str(Path.cwd().parent.absolute())

server_process = subprocess.Popen(
    [
        sys.executable,
        "-m",
        "uvicorn",
        "envs.echo_env.server.app:app",
        "--host",
        "0.0.0.0",
        "--port",
        "8001",
    ],
    env={**os.environ, "PYTHONPATH": f"{work_dir}/src"},
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    cwd=work_dir,
)

# Start background threads to stream errors
threading.Thread(
    target=stream_output, args=(server_process.stderr, "🔥 [stderr] "), daemon=True
).start()

print("⏳ Waiting for server to start...")
time.sleep(5)

try:
    response = requests.get("http://0.0.0.0:8001/health", timeout=2)
    print("\n✅ Echo Environment server is running!")
except Exception as e:
    print(f"\n❌ Server failed to start: {e}")
    print("\n📋 Checking error output...")
    server_process.poll()
    if server_process.stderr:
        stderr = server_process.stderr.read()
        if stderr:
            print(stderr)
    raise

try:
    input("Press Enter to exit...\n")
finally:
    print("🛑 Stopping server...")
    server_process.terminate()
    server_process.wait()

Overwriting openenv_tutorial/start_openenv_server.py


In [ ]:
import subprocess

# Start both servers in the background
server1 = subprocess.Popen(
    [
        "bash",
        "-c",
        (
            "CUDA_VISIBLE_DEVICES=0 trl vllm-serve "
            "--model Qwen/Qwen2.5-0.5B-Instruct "
            "--log-level warning "
            "--host 0.0.0.0 --port 8000"
        ),
    ]
)
server2 = subprocess.Popen(["python", f"{tutorial_dir}/start_openenv_server.py"])

print("Servers started. PIDs:", server1.pid, server2.pid)

Servers started. PIDs: 3616371 3616372


⚡ Starting FastAPI server for Echo Environment...
⏳ Waiting for server to start...


In [4]:
import time

import requests

URL = "http://0.0.0.0:8000/health"


def check_vllm_health():
    """Checks if the vLLM server is healthy."""
    try:
        response = requests.get(URL, timeout=3)
        if response.status_code == 200:
            print("✅ vLLM server is healthy!")
            return True
        else:
            print(f"⚠️ Server responded with {response.status_code}")
    except requests.RequestException as e:
        print(f"❌ Server not ready: {e}")
    return False


max_retries = 24
for attempt in range(1, max_retries + 1):
    if check_vllm_health():
        break
    time.sleep(5)
else:
    print(f"❌ Failed to start vLLM server after {max_retries} attempts.")

❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x725828b5b510>: Failed to establish a new connection: [Errno 111] Connection refused'))
🔥 [stderr] INFO:     Started server process [3616373]
🔥 [stderr] INFO:     Waiting for application startup.
🔥 [stderr] INFO:     Application startup complete.
🔥 [stderr] INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)

✅ Echo Environment server is running!
Press Enter to exit...
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x725828b61dd0>: Failed to establish a new connection: [Errno 111] Connection refused'))
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connecti

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-30 20:09:21 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM
INFO 10-30 20:09:21 [__init__.py:1815] Using max model len 32768
INFO 10-30 20:09:22 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=16384.
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x725828b69250>: Failed to establish a new connection: [Errno 111] Connection refused'))
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x725828b63110>: Failed to establish a new connection: [Errno 111] Connection refused'))
INFO 10-30 20:09:30 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:31 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=3616607) INFO 

[W1030 20:09:32.557912363 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:32 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=3616607) WARNING 10-30 20:09:32 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:32 [gpu_model_ru

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.46it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  6.45it/s]
(EngineCore_DP0 pid=3616607) 


(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:33 [default_loader.py:268] Loading weights took 0.17 seconds
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:33 [gpu_model_runner.py:2392] Model loading took 0.9266 GiB and 0.518918 seconds
❌ Server not ready: HTTPConnectionPool(host='0.0.0.0', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x725828b5ab10>: Failed to establish a new connection: [Errno 111] Connection refused'))
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:37 [backends.py:539] Using cache directory: /home/wizeng/.cache/vllm/torch_compile_cache/5d31f4c583/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:37 [backends.py:550] Dynamo bytecode transform time: 3.41 s
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:39 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.464 s
❌ Server not ready: HTTPConnectionPool(host='0.0.0

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:01<00:00, 40.61it/s]


(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:42 [gpu_model_runner.py:3118] Graph capturing finished in 2 secs, took 0.50 GiB
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:42 [gpu_worker.py:391] Free memory on device (78.59/79.19 GiB) on startup. Desired GPU memory utilization is (0.9, 71.27 GiB). Actual usage is 0.93 GiB for weight, 5.57 GiB for peak activation, 0.07 GiB for non-torch memory, and 0.5 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=68779733708` to fit into requested memory, or `--kv-cache-memory=76635612672` to fully utilize gpu memory. Current kv cache memory in use is 69478085324 bytes.
(EngineCore_DP0 pid=3616607) INFO 10-30 20:09:42 [core.py:218] init engine (profile, create kv cache, warmup model) took 8.65 seconds
INFO 10-30 20:09:43 [llm.py:295] Supported_tasks: ['generate']
INFO 10-30 20:09:43 [__init__.py:36] No IOProcessor plugins requested by the model
✅ vLLM server is healthy!


# Train the model!

By providing a custom rollout function to interact with the OpenEnv and vLLM servers, we can use trl to do agentic GRPO training. We also need to provide a reward function that processes the reward value output by the environment.

In [5]:
!tail -n +27 ../src/oumi/datasets/grpo/rollouts/echo_env_vllm_rollout.py

@register("echo_env_vllm_rollout", RegistryType.ROLLOUT_FUNCTION)
def echo_env_vllm_rollout(
    prompts: list[str], args, processing_class
) -> dict[str, list]:
    """Custom rollout function that generates completions via vLLM server and computes environment rewards.

    Args:
        prompts: List of prompts to generate from
        args: GRPOConfig containing all sampling parameters
        processing_class: Tokenizer/processor for decoding completions

    Returns:
        Dict containing prompt_ids, completion_ids, logprobs, and env_reward
    """  # noqa: E501
    # 1. Generate completions via vLLM inference server (running on port 8000)
    payload = {
        "prompts": prompts,
        "n": args.num_generations,
        "temperature": args.temperature,
        "top_p": args.top_p,
        "top_k": -1 if args.top_k is None else args.top_k,
        "min_p": 0.0 if args.min_p is None else args.min_p,
        "max_tokens": args.max_completion_length,
        "repetition_penalty"

In [6]:
!tail -n +23 ../src/oumi/datasets/grpo/rewards/env_reward.py

@register("env_reward", RegistryType.REWARD_FUNCTION)
def reward_from_env(completions, **kwargs):
    """Reward function that uses the environment reward."""
    # Extract environment rewards from kwargs (propagated via extra_fields)
    env_rewards = kwargs.get("env_reward", [])
    if env_rewards:
        return [float(reward) for reward in env_rewards]
    else:
        # Fallback if env_reward is not available
        return [0.0] * len(completions)


In [7]:
%%writefile $tutorial_dir/grpo_train.yaml

model:
  model_name: "Qwen/Qwen2-0.5B-Instruct"
  model_max_length: 2048
  torch_dtype_str: "bfloat16"
  attn_implementation: "sdpa"

data:
  train:
    datasets:
      - dataset_name: "trl-lib/ultrafeedback-prompt"
        split: "train"
        sample_count: 100

training:
  trainer_type: "TRL_GRPO"
  per_device_train_batch_size: 8
  gradient_accumulation_steps: 4

  reward_functions: ["env_reward"]

  ddp_find_unused_parameters: False
  optimizer: "adamw_torch_fused"

  grpo:
    use_vllm: True
    rollout_function: "echo_env_vllm_rollout"

  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32

  num_train_epochs: 1
  logging_steps: 1
  log_model_summary: False
  output_dir: "openenv_tutorial/echo_grpo"

Overwriting openenv_tutorial/grpo_train.yaml


In [8]:
!CUDA_VISIBLE_DEVICES=1 oumi train -c $tutorial_dir/grpo_train.yaml


   ____  _    _ __  __ _____
  / __ \| |  | |  \/  |_   _|
 | |  | | |  | | \  / | | |
 | |  | | |  | | |\/| | | |
 | |__| | |__| | |  | |_| |_
  \____/ \____/|_|  |_|_____|

⠸ Loading configuration...0m
[10/30/25 20:09:52] WARNING  [rank-0] Ignored             ]8;id=473956;file:///home/wizeng/repos/oumi/src/oumi/core/configs/training_config.py\training_config.py]8;;\:]8;id=913491;file:///home/wizeng/repos/oumi/src/oumi/core/configs/training_config.py#149\149]8;;\
                             model.model_max_length=2048                        
                             parameter for trainer                              
                             TrainerType.TRL_GRPO.                              
                    INFO     [rank-0] Setting random seed to  ]8;id=358934;file:///home/wizeng/repos/oumi/src/oumi/core/distributed.py\distributed.py]8;;\:]8;id=692189;file:///home/wizeng/repos/oumi/src/oumi/core/distributed.py#616\616]8;;\
                             4

Processed prompts: 100%|██████████| 32/32 [00:01<00:00, 27.97it/s, est. speed input: 1084.01 toks/s, output: 3418.09 toks/s]


{'loss': -0.3104, 'grad_norm': 5.6875, 'learning_rate': 5e-05, 'num_tokens': 5150.0, 'completions/mean_length': 122.1875, 'completions/min_length': 22.0, 'completions/max_length': 256.0, 'completions/clipped_ratio': 0.15625, 'completions/mean_terminated_length': 97.40740966796875, 'completions/min_terminated_length': 22.0, 'completions/max_terminated_length': 244.0, 'rewards/reward_from_env/mean': 60.24374771118164, 'rewards/reward_from_env/std': 46.80415344238281, 'reward': 60.24374771118164, 'reward_std': 20.179214477539062, 'frac_reward_zero_std': 0.0, 'sampling/sampling_logp_difference/mean': 0.10425987094640732, 'sampling/sampling_logp_difference/max': 1.4170303344726562, 'sampling/importance_sampling_ratio/min': 0.24243289232254028, 'sampling/importance_sampling_ratio/mean': 1.0240256786346436, 'sampling/importance_sampling_ratio/max': 1.5776448249816895, 'entropy': 1.3606750071048737, 'clip_ratio/low_mean': 0.0, 'clip_ratio/low_min': 0.0, 'clip_ratio/high_mean': 0.0, 'clip_ratio

Processed prompts: 100%|██████████| 32/32 [00:00<00:00, 43.54it/s, est. speed input: 3124.69 toks/s, output: 9436.59 toks/s]


{'loss': -0.1341, 'grad_norm': 4.28125, 'learning_rate': 4.8e-05, 'num_tokens': 14380.0, 'completions/mean_length': 216.6875, 'completions/min_length': 3.0, 'completions/max_length': 256.0, 'completions/clipped_ratio': 0.65625, 'completions/mean_terminated_length': 141.63636779785156, 'completions/min_terminated_length': 3.0, 'completions/max_terminated_length': 247.0, 'rewards/reward_from_env/mean': 99.38749694824219, 'rewards/reward_from_env/std': 42.04217529296875, 'reward': 99.38749694824219, 'reward_std': 17.43398094177246, 'frac_reward_zero_std': 0.0, 'sampling/sampling_logp_difference/mean': 0.0798419937491417, 'sampling/sampling_logp_difference/max': 1.7389039993286133, 'sampling/importance_sampling_ratio/min': 0.17571288347244263, 'sampling/importance_sampling_ratio/mean': 1.0187627077102661, 'sampling/importance_sampling_ratio/max': 1.660508632659912, 'entropy': 1.0239375233650208, 'clip_ratio/low_mean': 0.0, 'clip_ratio/low_min': 0.0, 'clip_ratio/high_mean': 0.0, 'clip_ratio

...

{'loss': -0.0005, 'grad_norm': 1.0, 'learning_rate': 6e-06, 'num_tokens': 226332.0, 'completions/mean_length': 256.0, 'completions/min_length': 256.0, 'completions/max_length': 256.0, 'completions/clipped_ratio': 1.0, 'completions/mean_terminated_length': 0.0, 'completions/min_terminated_length': 0.0, 'completions/max_terminated_length': 0.0, 'rewards/reward_from_env/mean': 299.62811279296875, 'rewards/reward_from_env/std': 2.361961603164673, 'reward': 299.62811279296875, 'reward_std': 1.885979175567627, 'frac_reward_zero_std': 0.0, 'sampling/sampling_logp_difference/mean': 0.004480584524571896, 'sampling/sampling_logp_difference/max': 1.2986259460449219, 'sampling/importance_sampling_ratio/min': 0.27290651202201843, 'sampling/importance_sampling_ratio/mean': 1.0008270740509033, 'sampling/importance_sampling_ratio/max': 1.284912347793579, 'entropy': 0.04254150390625, 'clip_ratio/low_mean': 0.0001220703125, 'clip_ratio/low_min': 0.0001220703125, 'clip_ratio/high_mean': 0.0, 'clip_ratio/

Processed prompts: 100%|██████████| 32/32 [00:00<00:00, 42.71it/s, est. speed input: 2178.70 toks/s, output: 10936.15 toks/s]


{'loss': -0.0, 'grad_norm': 1.1328125, 'learning_rate': 4.000000000000001e-06, 'num_tokens': 236156.0, 'completions/mean_length': 256.0, 'completions/min_length': 256.0, 'completions/max_length': 256.0, 'completions/clipped_ratio': 1.0, 'completions/mean_terminated_length': 0.0, 'completions/min_terminated_length': 0.0, 'completions/max_terminated_length': 0.0, 'rewards/reward_from_env/mean': 299.54376220703125, 'rewards/reward_from_env/std': 3.9939420223236084, 'reward': 299.54376220703125, 'reward_std': 2.759938955307007, 'frac_reward_zero_std': 0.0, 'sampling/sampling_logp_difference/mean': 0.004718138370662928, 'sampling/sampling_logp_difference/max': 1.1052160263061523, 'sampling/importance_sampling_ratio/min': 0.33113935589790344, 'sampling/importance_sampling_ratio/mean': 1.000596284866333, 'sampling/importance_sampling_ratio/max': 1.3233096599578857, 'entropy': 0.04345703125, 'clip_ratio/low_mean': 0.0, 'clip_ratio/low_min': 0.0, 'clip_ratio/high_mean': 0.0001220703125, 'clip_r

Processed prompts: 100%|██████████| 32/32 [00:00<00:00, 42.61it/s, est. speed input: 3100.11 toks/s, output: 10908.90 toks/s]


{'loss': -0.0003, 'grad_norm': 0.87109375, 'learning_rate': 2.0000000000000003e-06, 'num_tokens': 246676.0, 'completions/mean_length': 256.0, 'completions/min_length': 256.0, 'completions/max_length': 256.0, 'completions/clipped_ratio': 1.0, 'completions/mean_terminated_length': 0.0, 'completions/min_terminated_length': 0.0, 'completions/max_terminated_length': 0.0, 'rewards/reward_from_env/mean': 300.546875, 'rewards/reward_from_env/std': 1.7720036506652832, 'reward': 300.546875, 'reward_std': 1.5536911487579346, 'frac_reward_zero_std': 0.0, 'sampling/sampling_logp_difference/mean': 0.003943466581404209, 'sampling/sampling_logp_difference/max': 0.8658556938171387, 'sampling/importance_sampling_ratio/min': 0.4206914007663727, 'sampling/importance_sampling_ratio/mean': 1.0006545782089233, 'sampling/importance_sampling_ratio/max': 1.237608790397644, 'entropy': 0.037139892578125, 'clip_ratio/low_mean': 0.0, 'clip_ratio/low_min': 0.0, 'clip_ratio/high_mean': 0.0, 'clip_ratio/high_max': 0.0